# MVSep-MDX23 Colab Fork v2.2.1-gui
Colab version of MDX23 algorithm from [MVSEP.COM](https://www.mvsep.com) with some tweaks:
<font size=2>

**v2.2.1-gui**
* Added semi-working gui.py for those brave souls, who wish infering audio without Jupyter notebooks
* Added file upload feature in Colab
* Added russian translation for Colab ([русская версия здесь](https://colab.research.google.com/github/alphatoasterous/MVSEP-MDX23-Colab_v2/blob/v2.2/MVSep-MDX23-Colab-Russian.ipynb))

**v2.2.1**
* Added custom weights feature
* Fixed some bugs
* Fixed input: you can use a file or a folder as input now

**v2.2**
* Added MDXv3 compatibility 
* Added MDXv3 demo model D1581 in vocals stem multiband ensemble.
* Added VOC-FT Fullband SRS instead of UVR-MDX-Instr-HQ3.
* Added 2stems feature : output only vocals/instrum (faster processing)
* Added 16bit output format option
* Added "BigShift trick" for MDX models
* Added separated overlap values for MDX, MDXv3 and Demucs
* Fixed volume compensation fine-tuning for MDX-VOC-FT
<br>

**v2.1 (by deton24)**
* Updated with MDX-VOC-FT instead of Kim Vocal 2
<br>

**v2.0**
* Updated with new Kim Vocal 2 & UVR-MDX-Instr-HQ3 models
* Folder batch processing
* Fixed high frequency bleed in vocals
* Fixed volume compensation for MDX models
<br>

Credits:
* [ZFTurbo/MVSep](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Models by [Demucs](https://github.com/facebookresearch/demucs), [Anjok](https://github.com/Anjok07/ultimatevocalremovergui) & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Adaptation & tweaks by [jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)
* Standalone GUI, Russian translation and upload feature bodged together by [alphatoaster](https://github.com/alphatoasterous/MVSEP-MDX23-Colab_v2)
</font>


In [ ]:
#@markdown #Installation
#@markdown *Run this cell to install MVSep-MDX23*
print('Installing... This will take 1 minute...')
%cd /content
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2
!pip install -r requirements.txt &> /dev/null
print('Installation done !')

In [ ]:
#@markdown #File upload
#@markdown *Run this cell to upload your audio files(WAV, FLAC, MP3 or zip files containing audio) into this Colab. <br> Taken and bodged in from [this Colab](https://colab.research.google.com/drive/1TU-kkQWVf-PLO_hSa2QCMZS1XF5xVHqs) with slight adjustments*

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Create the target directory if it doesn't exist
target_dir = '/content/audio_files/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Check if the uploaded file is a zip file
    if fn.endswith('.zip'):
        # Write the uploaded zip file to the target directory
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Delete the zip file
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Zip file "{name}" extracted and removed. Files are in: {folder}'.format(name=fn, folder=unzip_dir))

        # Display copy path buttons for each extracted file
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Extracted file "{name}" with length {length} bytes'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='File saved to: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path  # Make a local copy of the file path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # For non-zip files
    # Save the file to the target directory
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Check if the uploaded file is a .pth or .index file
    if not (fn.endswith('.wav') or fn.endswith('.mp3') or fn.endswith('.flac')):
        warning_text = widgets.HTML(
            value='<b style="color: red;">Warning:</b> You are uploading an unsupported file.'
        )
        display(warning_text)

    # Create a clickable path with copy button
    file_path_text = widgets.HTML(
        value='File saved to: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

# Remove the original uploaded files from /content/
for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

In [ ]:
#@markdown #Separation
#@markdown *Copy path provided in previous code cell, or leave it as is for batch processing*

from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2


input = '/content/audio_files/' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
#@markdown ---
#@markdown <font size=2>*Bigshifts=1 to disable that feature*</font>
BigShifts_MDX = 11 #@param {type:"slider", min:1, max:41, step:1}
overlap_MDX = 0 #@param {type:"slider", min:0, max:0.95, step:0.05}
overlap_MDXv3 = 8 #@param {type:"slider", min:2, max:40, step:2}
#@markdown ---
weight_MDXv3 = 6 #@param {type:"slider", min:0, max:10, step:1}
weight_VOCFT = 5 #@param {type:"slider", min:0, max:10, step:1}
weight_HQ3 = 2 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
output_format = 'PCM_16' #@param ["PCM_16", "FLOAT"]
vocals_instru_only = True #@param {type:"boolean"}
if vocals_instru_only:
    vocals_only = '--vocals_only true'
else:
    vocals_only = ''

#@markdown <font size=2 color=red>Use lower chunk_size value if you have memory errors !</font>
chunk_size = 500000 #@param {type:"slider", min:100000, max:1000000, step:100000}

if Path(input).is_file():
  file_path = input
  filename =  Path(input).stem
  Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --large_gpu \
        --weight_MDXv3 {weight_MDXv3} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_HQ3 {weight_HQ3} \
        --chunk_size {chunk_size} \
        --input_audio "{file_path}" \
        --overlap_demucs {overlap_demucs} \
        --overlap_MDX {overlap_MDX} \
        --overlap_MDXv3 {overlap_MDXv3} \
        --output_format {output_format} \
        --bigshifts {BigShifts_MDX} \
        --output_folder "{output_folder}/{filename}" \
        {vocals_only}
else:  
  for file_path in sorted(glob.glob(input+"/*"))[:]:
    filename =  Path(file_path).stem
    Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
    !python inference.py \
        --large_gpu \
        --weight_MDXv3 {weight_MDXv3} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_HQ3 {weight_HQ3} \
        --chunk_size {chunk_size} \
        --input_audio "{file_path}" \
        --overlap_demucs {overlap_demucs} \
        --overlap_MDX {overlap_MDX} \
        --overlap_MDXv3 {overlap_MDXv3} \
        --output_format {output_format} \
        --bigshifts {BigShifts_MDX} \
        --output_folder "{output_folder}/{filename}" \
        {vocals_only}